<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [1]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [45]:
# general python packages
import time
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import pandas
pandas.set_option('display.max_rows', 1000)

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
import lsst.daf.butler as Butler

# LSST package for image display
import lsst.afw.display as afwDisplay
import lsst.geom

from astropy import units as u
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import gc

# Astropy imports
from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb

import lsst.geom as geom
# Object for multi-band exposures
from lsst.afw.image import MultibandExposure

plt.style.use('tableau-colorblind10')
%matplotlib inline

### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

## 2. Search data for a supernova for which to create image cutouts

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading

In [3]:
service = get_tap_service()
assert service is not None

In [4]:
# galaxy cluster coords: 55.6521739130, -31.9834710744
# grab a DiaObject identified in Tutorial 07a to be a candidate Type 1a SN with relatively few number of 
# nDiaSources (frames over which flux varies): RA=51.2332445 Dec=-44.509536 DiaObjID=1248649384967537762
#%%time

# First setup the search like in tutorial 07a:
redshift_min = 0.1
redshift_max = 0.3
snia_peak_mag = -19.0
snia_peak_mag_range = 0.5
snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0
snia_ampl_mr_min = 1.5
snia_ampl_mr_max = 5.5
snia_peak_mr_min = 18.82
snia_peak_mr_max = 22.46
# pick a set on the smaller end of time to make the output cutouts managable:
snia_nDiaSources_min = 15
snia_nDiaSources_max = 20

# Print the contents of the catalog so you see what you can access
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
#                         "WHERE table_name = 'dp02_dc2_catalogs.DiaObject'")
                         "WHERE table_name = 'dp02_dc2_catalogs.DiaSource'")
#results.to_table().to_pandas()
#according to the DPDD, the DiaSource tables should have the ccdVisitId
#results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
#                         "WHERE table_name = 'dp02_dc2_catalogs.Object'")
#results.to_table().to_pandas()


In [5]:
%%time

results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         #"visinfo.ccdVisitId AS visit, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin) AS rMagMax, "
                         "scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagMin, "
                         "scisql_nanojanskyToAbMag(gPSFluxMax) AS gMagMin, "
                         "scisql_nanojanskyToAbMag(iPSFluxMax) AS iMagMin, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagAmp "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE nDiaSources > "+str(snia_nDiaSources_min)+" "
                         "AND nDiaSources < "+str(snia_nDiaSources_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(gPSFluxMax) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_nanojanskyToAbMag(iPSFluxMax) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_ampl_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_ampl_mr_min)+" ",
                         maxrec=1000)

#to use visitinfo you have to add to end of query:
#"ON visinfo.ccdVisitId = src.ccdVisitId "\
#        "WHERE src.objectId = "+str(sel_objid)+" "

# Can you get visit # added to this?

DiaObjs = results.to_table()
del results

CPU times: user 47.7 ms, sys: 3.43 ms, total: 51.1 ms
Wall time: 2min 37s


In [6]:
#display the table
DiaObjs

ra,decl,diaObjectId,nDiaSources,rMagMax,rMagMin,gMagMin,iMagMin,rMagAmp
deg,deg,,,,,,,
float64,float64,int64,int64,float64,float64,float64,float64,float64
67.9203038,-35.3190375,1737923263234837021,17,20.766806288265464,18.99597475660502,21.339931744633297,21.964724845387174,1.7708315316604448
59.9380586,-34.7474072,1735917754025772373,16,22.864381695706435,19.111948731826047,19.58794322163218,19.777727876546486,3.7524329638803877
63.1203751,-35.0660157,1736726994583816852,19,21.290769351589145,19.234241185466097,16.682591880334144,17.27430140266422,2.056528166123048
62.7007446,-35.2198161,1736683014118703808,17,22.898745496420077,19.896018440842575,21.04134439838197,21.36676835905962,3.002727055577502
66.0686493,-34.5553986,1737738545281368866,19,23.59666055025863,21.826947123548365,23.819684714328744,22.299079721011424,1.7697134267102648
66.3935018,-34.7163452,1737668176537190778,16,22.438569033456446,19.814189114748544,19.373567777066924,19.194039883195952,2.624379918707902
66.5305378,-34.9292552,1737597807793012974,18,25.5653580885755,22.065806066470884,22.604783496155783,21.729098240466094,3.4995520221046164
67.5669884,-34.6625872,1738116777281323055,16,23.0792464048035,21.47059677725551,21.541450739432797,21.45866914326122,1.6086496275479902


In [7]:
# Initialize the Butler  
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler.Butler(config, collections=collection)
registry = butler.registry

In [21]:
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp02_dc2_catalogs.DiaSource'")
results.to_table().to_pandas()

,column_name,datatype,description,unit
0,apFlux,double,Flux within 12.0-pixel aperture,nJy
1,apFlux_flag,boolean,General Failure Flag,
2,apFlux_flag_apertureTruncated,boolean,Aperture did not fit within measurement image,
3,apFluxErr,double,Flux uncertainty within 12.0-pixel aperture,nJy
4,bboxSize,long,Bounding box of diaSource footprint,
5,ccdVisitId,long,Unique ID of CCD and visit where this source w...,
6,centroid_flag,boolean,"General failure flag, set if anything went wrong",
7,centroid_neg_flag,boolean,"Failure flag for negative, set if anything wen...",
8,centroid_pos_flag,boolean,"Failure flag for positive, set if anything wen...",
9,coord_dec,double,Fiducial ICRS Declination of centroid used for...,deg


In [37]:
query = "SELECT TOP " + str(10) + " " + \
        "diaObjectId, ra, decl, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " #+ \

results = service.search(query)
results.to_table().to_pandas()

,diaObjectId,ra,decl,ccdVisitId
0,1249546586455801988,53.967483,-44.488183,941549088
1,1249546586455801988,53.967564,-44.488106,1169966169
2,1249546586455801988,53.967451,-44.488129,945613109
3,1249546586455801988,53.967523,-44.488120,645665103
4,1249546586455801988,53.967433,-44.488242,645686129
5,1249546586455801988,53.967559,-44.488148,1165969067
6,1249546586455801988,53.967436,-44.488231,1165727035
7,1249546586455801988,53.967500,-44.488156,479655048
8,1249546586455801988,53.967495,-44.488163,410029055
9,1249546586455801988,53.967457,-44.488241,1165696084


In [36]:
# just pick a source to use for the cutouts, like the first one
ra, dec = DiaObjs['ra'][0], DiaObjs['decl'][0]
sel_objid = DiaObjs['diaObjectId'][0]
print(sel_objid)

center_coords = SkyCoord(ra, dec, frame='icrs', unit='deg')
search_radius = 1.0*u.arcmin
use_center_coords = str(ra)+", "+str(dec) #"62, -37"
use_radius = "0.00028" #1 arcsec in deg

print(center_coords)
print(search_radius)
                
# find the source using a small search radius, but there are too many DiaSources in a small area (weird)
query = "SELECT TOP " + str(20) + " " + \
        "diaObjectId, ra, decl, coord_ra, coord_dec, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " + \
        "WHERE CONTAINS(POINT('ICRS', ra, decl), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " 
        
visits = service.search(query)
visits.to_table().to_pandas()

1737923263234837021
<SkyCoord (ICRS): (ra, dec) in deg
    (67.9203038, -35.3190375)>
1.0 arcmin


,diaObjectId,ra,decl,coord_ra,coord_dec,ccdVisitId
0,1737923263234834581,67.920119,-35.319042,67.920027,-35.319081,944250012
1,1737923263234834581,67.920079,-35.319139,67.920027,-35.319081,1176304153
2,1737923263234834581,67.920038,-35.319141,67.920027,-35.319081,246710025
3,1737923263234834581,67.919981,-35.319049,67.920027,-35.319081,12464067
4,1737923263234834581,67.920021,-35.319019,67.920027,-35.319081,282400171
5,1737923263234834581,67.920063,-35.319158,67.920027,-35.319081,516406122
6,1737923263234837021,67.920271,-35.319064,67.920304,-35.319038,1196460149
7,1737923263234837021,67.920317,-35.319007,67.920304,-35.319038,913756017
8,1737923263234837021,67.920302,-35.318977,67.920304,-35.319038,1218488178
9,1737923263234837021,67.920276,-35.319040,67.920304,-35.319038,1195351054


In [47]:
# find the source by matching the source ID, which gives you the exact source
query = "SELECT TOP " + str(100) + " " + \
        "diaObjectId, ra, decl, coord_ra, coord_dec, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " + \
        "WHERE diaObjectId = " + str(sel_objid) + " "

visits = service.search(query)
visits.to_table().to_pandas()

,diaObjectId,ra,decl,coord_ra,coord_dec,ccdVisitId
0,1737923263234837021,67.920271,-35.319064,67.920304,-35.319038,1196460149
1,1737923263234837021,67.920317,-35.319007,67.920304,-35.319038,913756017
2,1737923263234837021,67.920302,-35.318977,67.920304,-35.319038,1218488178
3,1737923263234837021,67.920276,-35.319040,67.920304,-35.319038,1195351054
4,1737923263234837021,67.920294,-35.319063,67.920304,-35.319038,1189138096
5,1737923263234837021,67.920343,-35.319106,67.920304,-35.319038,1160446017
6,1737923263234837021,67.920305,-35.319082,67.920304,-35.319038,480680008
7,1737923263234837021,67.920287,-35.319057,67.920304,-35.319038,690970011
8,1737923263234837021,67.920290,-35.319039,67.920304,-35.319038,753022056
9,1737923263234837021,67.920319,-35.319047,67.920304,-35.319038,1225386161


In [43]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.
    
    CCW snagged this routine from tutorial notebook 03a

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image


In [46]:
# now try to make the cutout using butler
radec = lsst.geom.SpherePoint(ra*lsst.geom.degrees,dec*lsst.geom.degrees)

skymap = butler.get('skyMap')
#cutout = cutout_coadd(butler, ra, dec, band='g', datasetType='deepCoadd', cutoutSideLength=1000)

# To get the visits used to contruct the deepCoadd
#unique_visits = list(set([i['visit'] for i in cutout.getInfo().getCoaddInputs().ccds]))


# Look up the tract, patch for the RA, Dec
tractInfo = skymap.findTract(radec)
my_patch = tractInfo.findPatch(radec)
dataId = {'visit': visits['ccdVisitId'][0], 'band': 'i', 'tract': tractInfo, 'patch': my_patch}

print(butler.registry.getDatasetType('calexp'))


# Select a position at roughly the center of the galaxy cluster:
cutout_image = cutout_coadd(butler, ra, dec, band='i', datasetType='calexp',
                            cutoutSideLength=50)

fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout_image.image)
plt.show()
remove_figure(fig)


# deep coadd, we dont' want this because we want the individual time slices
#my_deepCoadd = butler.get('deepCoadd', dataId=dataId)
# datasetType of calexp = processed visit images, we want this for time series
#exposures = butler.get('calexp', dataId=dataId)

#xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
#bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
#patch = tractInfo.getSequentialPatchIndex(patchInfo)

#coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
#parameters = {'bbox': bbox}

#cutout_image = butler.get(datasetType, parameters=parameters,
#                           dataId=coaddId)

DatasetType('calexp', {band, instrument, detector, physical_filter, visit_system, visit}, ExposureF)


KeyError: "No value in data ID ({'tract': 4032, 'patch': 11, 'band': 'i'}) for required dimension 'detector'."